# Hyperopt parameters leafmodels

#### Feature Selection BPIC2015 2 f2

In [39]:
#12 columns (20% correlation)

In [40]:
columns_PC20= ['agg__Activity_08_AWB45_020_2',
 'static__Milieu (vergunning)',
 'agg__question_28',
 'agg__monitoringResource_4634935',
 'agg__Activity_08_AWB45_025',
 'agg__question_Uitgebreid',
 'agg__Activity_08_AWB45_170',
 'agg__Activity_01_HOOFD_330',
 'agg__org:resource_560530',
 'agg__org:resource_4634935',
 'agg__Activity_01_HOOFD_193',
 'agg__question_other']

In [41]:
#22 columns (15% correlation)

In [42]:
columns_PC15= ['agg__Activity_08_AWB45_020_2',
 'static__Milieu (vergunning)',
 'agg__question_28',
 'agg__monitoringResource_4634935',
 'agg__Activity_08_AWB45_025',
 'agg__question_Uitgebreid',
 'agg__Activity_08_AWB45_170',
 'agg__Activity_01_HOOFD_330',
 'agg__org:resource_560530',
 'agg__org:resource_4634935',
 'agg__Activity_01_HOOFD_193',
 'agg__question_other',
 'agg__Activity_08_AWB45_060',
 'agg__Activity_01_HOOFD_191',
 'agg__Activity_08_AWB45_051_0',
 'agg__timesincelastevent_sum',
 'agg__Activity_01_HOOFD_490_1',
 'agg__Activity_01_HOOFD_250',
 'agg__Activity_08_AWB45_090_2',
 'static__Kap',
 'static__Milieu (neutraal wijziging)',
 'agg__Activity_01_HOOFD_490_2']

In [43]:
#50 columns (10% correlation)

In [44]:
columns_PC10= ['agg__Activity_08_AWB45_020_2',
 'static__Milieu (vergunning)',
 'agg__question_28',
 'agg__monitoringResource_4634935',
 'agg__Activity_08_AWB45_025',
 'agg__question_Uitgebreid',
 'agg__Activity_08_AWB45_170',
 'agg__Activity_01_HOOFD_330',
 'agg__org:resource_560530',
 'agg__org:resource_4634935',
 'agg__Activity_01_HOOFD_193',
 'agg__question_other',
 'agg__Activity_08_AWB45_060',
 'agg__Activity_01_HOOFD_191',
 'agg__Activity_08_AWB45_051_0',
 'agg__timesincelastevent_sum',
 'agg__Activity_01_HOOFD_490_1',
 'agg__Activity_01_HOOFD_250',
 'agg__Activity_08_AWB45_090_2',
 'static__Kap',
 'static__Milieu (neutraal wijziging)',
 'agg__Activity_08_AWB45_070_3',
 'static__Gebiedsbescherming',
 'agg__month_std',
 'agg__timesincecasestart_std',
 'agg__Activity_01_HOOFD_250_1',
 'agg__question_42',
 'agg__Activity_01_HOOFD_130',
 'agg__Activity_01_HOOFD_250_2',
 'agg__weekday_std',
 'agg__Activity_08_OLO_100',
 'agg__Activity_01_HOOFD_190_1',
 'static__Responsible_actor_560530',
 'agg__Activity_11_AH_II_010',
 'agg__Activity_08_AWB45_050',
 'agg__timesincecasestart_mean',
 'agg__weekday_max',
 'agg__timesincelastevent_max',
 'agg__Activity_08_AWB45_090_1',
 'agg__timesincelastevent_std',
 'agg__Activity_08_AWB45_020_0',
 'agg__timesincecasestart_sum',
 'agg__open_cases_std',
 'agg__timesincelastevent_mean',
 'agg__Activity_01_HOOFD_190_2',
 'static__Inrit/Uitweg',
 'agg__Activity_01_HOOFD_470',
 'agg__Activity_01_HOOFD_510_2',
 'agg__month_min',
 'agg__Activity_01_HOOFD_495']

#### Feature Selection Traffic_Fines_1

In [23]:
#15% PC

In [24]:
columns_PC15 = ['agg__Activity_Payment', 'agg__Activity_Send Fine']

In [25]:
#10% PC

In [26]:
columns_PC10 = ['agg__Activity_Payment', 'agg__Activity_Send Fine', 'agg__Activity_Insert Date Appeal to Prefecture']

#### Feature Selection BPIC2017

In [ ]:
#20% PC

In [30]:
columns_PC20= ['agg__CreditScore_std', 'agg__Selected_False']

In [31]:
#15% PC

In [32]:
columns_PC15= ['agg__CreditScore_std',
 'agg__Selected_False',
 'static__ApplicationType_New credit',
 'static__ApplicationType_Limit raise']

In [33]:
#10% PC

In [34]:
columns_PC10 = ['static__ApplicationType_Limit raise',
 'static__ApplicationType_New credit',
 'agg__Activity_A_Cancelled',
 'agg__Activity_A_Submitted',
 'agg__Activity_W_Validate application',
 'agg__lifecycle:transition_ate_abort',
 'agg__Accepted_False',
 'agg__Accepted_True',
 'agg__Selected_False',
 'agg__CreditScore_std']

### run

In [47]:
import pandas as pd
import numpy as np

import os
import random

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import FeatureUnion
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import pickle

import hyperopt
from hyperopt import hp, Trials, fmin, tpe, STATUS_OK
from hyperopt.pyll.base import scope

#user-specified packages from Benchmark for outcome-oriented predictive process monitoring (see acknowledgements)
import EncoderFactory
from DatasetManager import DatasetManager

def create_and_evaluate_model(args): 
    global trial_nr
    trial_nr += 1
    
    start = time.time()
    score = 0
    for cv_iter in range(n_splits):
        
        dt_test_prefixes = dt_prefixes[cv_iter]
        dt_train_prefixes = pd.DataFrame()
        for cv_train_iter in range(n_splits): 
            if cv_train_iter != cv_iter:
                dt_train_prefixes = pd.concat([dt_train_prefixes, dt_prefixes[cv_train_iter]], axis=0)
        
        #remove training rows with negative values (should be a mistake)
        #dt_train_prefixes = dt_train_prefixes[dt_train_prefixes.select_dtypes(include=[np.number]).ge(0).all(1)]

        preds_all = []
        test_y_all = []
        test_y = dataset_manager.get_label_numeric(dt_test_prefixes)
        train_y = dataset_manager.get_label_numeric(dt_train_prefixes)  
                 
        #feature combiner
        feature_combiner = FeatureUnion([(method, EncoderFactory.get_encoder(method, **cls_encoder_args)) for method in methods])
        feature_combiner.fit(dt_train_prefixes, train_y)

        #transform train dataset
        dt_train_named= feature_combiner.transform(dt_train_prefixes)
        dt_train_named = pd.DataFrame(dt_train_named)
        names= feature_combiner.get_feature_names()
        dt_train_named.columns = names
        dt_train_named = dt_train_named[columns]
        
        
        #first, create,train and fit a DecisionTreeClassifier
        cls = DecisionTreeClassifier(criterion= 'entropy', max_depth= args['max_depth'], min_samples_leaf= args['min_samples_leaf'], random_state = random_state)
        cls.fit(dt_train_named,train_y)
       
        #transform test dataset
        dt_test_named = feature_combiner.transform(dt_test_prefixes)
        dt_test_named = pd.DataFrame(dt_test_named)
        names= feature_combiner.get_feature_names()
        dt_test_named.columns = names
        
        
        dt_test_named = dt_test_named[columns]
        
        
        
        #homogeneuous segments (buckets)
        dt_train_named['cluster'] = cls.apply(dt_train_named)
        dt_test_named['cluster'] = cls.apply(dt_test_named)

        #temporarily concatenate back, in order to have the correct y-values per segment
        #train data 
        train_y_concat = pd.DataFrame(train_y)
        train_y_concat = train_y_concat.rename(columns={train_y_concat.columns[0]:'label'})
        dt_train_named = pd.concat([dt_train_named,train_y_concat], axis=1)
        
        #test data
        test_y_concat  = pd.DataFrame(test_y)
        test_y_concat  = test_y_concat.rename(columns={test_y_concat.columns[0]:'label'})
        dt_test_named = pd.concat([dt_test_named, test_y_concat], axis=1)
    
        #list of buckets
        buckets = list((dt_test_named['cluster'].unique()))
        for i in buckets: 
            #only take the data from the bucket, seperate the label from the independent features
            data_train_x = dt_train_named[dt_train_named['cluster']==i].copy()
            data_train_y = data_train_x['label'].copy()
    
            data_test_x  = dt_test_named[dt_test_named['cluster']==i].copy()
            data_test_y  = data_test_x['label'].copy()
        
            #drop the columns
            data_train_x = data_train_x.drop('label', axis=1)
            data_train_x = data_train_x.drop('cluster', axis=1)
            data_test_x = data_test_x.drop('label', axis=1)
            data_test_x = data_test_x.drop('cluster', axis=1)
        
            #if there is only one label in the training data, no need to create a leaf model
            if len(set(data_train_y))<2:
                pred = [data_train_y.iloc[0]]*len(data_test_y)
                preds_all.extend(pred)
                test_y_all.extend(data_test_y)
                
            else:  
                #print length of test and training data of the leaf node
                test_y_all.extend(data_test_y)
            
                if cls_method == 'llm':
                    scaler = StandardScaler()
                    data_train_x= scaler.fit_transform(data_train_x)
                    data_test_x= scaler.transform(data_test_x)
                    preds_pos_label_idx = np.where(cls.classes_ == 1)[0][0]
                    logreg = LogisticRegression(max_iter= 10000, 
                                            solver='sag', 
                                            n_jobs =-1)
                    logreg.fit(data_train_x, data_train_y)
                    pred = logreg.predict_proba(data_test_x)
                    pred = pred[:,preds_pos_label_idx]
                    preds_all.extend(pred)   

        score += roc_auc_score(test_y_all, preds_all)
    
    
       
        for k, v in args.items():
            fout_all.write("%s;%s;%s;%s;%s;%s;%s\n" % (trial_nr, dataset_name, cls_method, method_name, k, v, score / n_splits))   
        fout_all.write("%s;%s;%s;%s;%s;%s;%s\n" % (trial_nr, dataset_name, cls_method, method_name, "processing_time", time.time() - start, 0))   
    fout_all.flush()
    return {'loss': -score / n_splits, 'status': STATUS_OK, 'model': cls}

dataset_ref = 'bpic2015_2_f2'
params_dir = './params_dir'
n_iter = 5
cls_encoding = 'agg'
cls_method = 'llm'

column_selection = 'PC15'
columns = columns_PC15

method_name = "%s_%s"%(column_selection, cls_encoding)


dataset_ref_to_datasets = {
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(1,6)]
}

encoding_dict = {
    "agg": ["static", "agg"]
}

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]
methods = encoding_dict[cls_encoding]
    
train_ratio = 0.8
n_splits = 3
random_state = 22

# create results directory
if not os.path.exists(os.path.join(params_dir)):
    os.makedirs(os.path.join(params_dir))
    
for dataset_name in datasets:
    
    # read the data
    dataset_manager = DatasetManager(dataset_name)
    data = dataset_manager.read_dataset()
    cls_encoder_args = {'case_id_col': dataset_manager.case_id_col, 
                        'static_cat_cols': dataset_manager.static_cat_cols,
                        'static_num_cols': dataset_manager.static_num_cols, 
                        'dynamic_cat_cols': dataset_manager.dynamic_cat_cols,
                        'dynamic_num_cols': dataset_manager.dynamic_num_cols, 
                        'fillna': True}

    # determine min and max (truncated) prefix lengths
    min_prefix_length = 1
    if "traffic_fines" in dataset_name:
        max_prefix_length = 10
    elif "bpic2017" in dataset_name:
        max_prefix_length = min(20, dataset_manager.get_pos_case_length_quantile(data, 0.90))
    else:
        max_prefix_length = min(40, dataset_manager.get_pos_case_length_quantile(data, 0.90))

    # split into training and test
    train, _ = dataset_manager.split_data_strict(data, train_ratio, split="temporal")
    
    
    # prepare chunks for CV
    dt_prefixes = []
    class_ratios = []
    for train_chunk, test_chunk in dataset_manager.get_stratified_split_generator(train, n_splits=n_splits):
        class_ratios.append(dataset_manager.get_class_ratio(train_chunk))
        # generate data where each prefix is a separate instance
        dt_prefixes.append(dataset_manager.generate_prefix_data(test_chunk, min_prefix_length, max_prefix_length))
    del train
        
    # set up search space
    
    if cls_method == "llm":
        space = {"max_depth": scope.int(hp.quniform("max_depth",1,2,1)),
                 "min_samples_leaf": scope.int(hp.quniform("min_samples_leaf",100,1000,10))
                }
          
    # optimize parameters
    trial_nr = 0
    trials = Trials()
    fout_all = open(os.path.join(params_dir, "param_optim_all_trials_%s_%s_%s.csv" % (cls_method, dataset_name, method_name)), "w")
    if "prefix" in method_name:
        fout_all.write("%s;%s;%s;%s;%s;%s;%s;%s\n" % ("iter", "dataset", "cls", "method", "nr_events", "param", "value", "score"))   
    else:
        fout_all.write("%s;%s;%s;%s;%s;%s;%s\n" % ("iter", "dataset", "cls", "method", "param", "value", "score"))   
    rstate = np.random.RandomState(22)
    best = fmin(create_and_evaluate_model, space, algo=tpe.suggest, max_evals=n_iter, trials=trials, rstate = rstate)
    fout_all.close()

    # write the best parameters
    best_params = hyperopt.space_eval(space, best)
    outfile = os.path.join(params_dir, "optimal_params_%s_%s_%s.pickle" % (cls_method, dataset_name, method_name))
    # write to file
    with open(outfile, "wb") as fout:
        pickle.dump(best_params, fout)

100%|█████████████████████████████████████████████████| 5/5 [01:24<00:00, 16.96s/trial, best loss: -0.9508083448421693]
